## 11.인터페이스 : 프로토콜에서 ABC까지
### 11.1 파이썬 문화에서의 인터페이스와 프로토콜

파이썬에는 interface라는 키워드는 없지만 ABC에 상관없이 모든 클래스는 인터페이스를 가지고 있다. 클래스가 상속하거나 구현한 공개속성들의 집합이 인터페이스다. 여기에는 \_\_getitem\_\_( ), \_\_add\_\_( )와 같은 특별 메서드도 포함된다. 보호된 속성은 단지 명명관례(앞에 언더바를 하나 붙임)일 뿐이지만 인터페이스에 속하지 않는다는 관례를 어기는 것은 좋지 않다. 

한편 공개 데이터 속성을 객체의 인터페이스로 사용하는 것은 좋은 방법이다. 필요하면 언제나 데이터 속성을 호출 코드의 손상없이 <객체>, <속성> 구문을 사용해서 게터/세터를 구현하는 프로퍼티로 변환할 수 있기 때문이다. 

인터페이스의 정의에 대해서는 '시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드'라고 설명할 수 있다. 스몰토크에서는 프로토콜이라 불렀다. 프로코톨은 그래서 상속과 무관하다.

In [1]:
def extends_func(klass):
    def _(func):
        setattr(klass, func.__name__, func)
        return func
    return decorator

In [2]:
""" [예제 11-1] x, y는 공개 데이터 속성이다. """
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [3]:
v1 = Vector2d(1.1, 2.2)
print(v1.x)
v1.x = 3.3
print(v1.x)

1.1
3.3


아래는 x, y를 읽기 전용 프로퍼티로 변경한 내용이다. 사용자는 여전히 my_vector.x와 my_vector.y를 이용해서 값을 읽을 수 있다.

In [4]:
""" [예제 11-2] 프로퍼티로 다시 구현한 x와 y, """
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property    
    def x(self):
        return self.__x
    
    @property    
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [5]:
v1 = Vector2d(1.1, 2.2)
print(v1.x)
print(v1.__dict__)
v1._Vector2d__x = 3.3
print(v1.x)
v1.x = 3.3 # 읽기 전용 프로퍼티는 변경할 수 없다. 
print(v1.x)

1.1
{'_Vector2d__x': 1.1, '_Vector2d__y': 2.2}
3.3


AttributeError: can't set attribute

### 11.2 파이썬은 시퀀스를 찾아낸다. 

collections.abc에 정의된 Sequence ABC 및 관련 추상 클래스에 대한 UML 다이어그램. 상속 화살표는 서브클래스에서 슈퍼클래스를 가리킨다. 이탤릭체는 추상 메서드나 추상 클래스를 나타낸다. 

<img src="Figure11-1.png" alt="Trulli" width="500" height="333">

아래 클래스는 abc.Sequence를 상속하지 않으며, 시퀀스 프로토콜 메서드 중 \_\_getitem\_\_( ) 하나만 구현한다. ※ \_\_len\_\_( )은 미구현

\_\_iter\_\_( ) 메서드는 구현하지 않았지만 대체 수단인 \_\_getitem\_\_( ) 메서드가 구현되어 있으므로 Foo를 반복할 수 있으며, \_\_contains\_\_( ) 메서드가 구현되어 있지 않더라도 객체 전체를 조사해서 항목을 찾아냄으로써 in 연산자도 동작 시킬 수 있다. 결국 객체를 반복하기 위해 파이썬 인터프리터는 두 가지 다른 메서드를 시도하므로, 반복은 덕 타이핑의 극단적인 예를 보여준다. 

In [6]:
""" [예제 11-4] 일련의 카드로 구성한 카드 한 벌 """

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [
            Card(rank, suit) for suit in self.suits
            for rank in self. ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position): # 시퀀스 프로토콜의 중요성 때문에 __iter__(), contains__() 메서드 구현 없이
        return self._cards[position] # __getitem__() 구현만으로 반복할 수 있으며, in 연산자에서 사용할 수 있다.
    
from types import FunctionType
[x for x, y in FrenchDeck.__dict__.items() if type(y) == FunctionType] # Class member function
                                                                       # 멍키 패칭 전에는 __setitem__() 맴버 함수가 없음

['__init__', '__len__', '__getitem__']

### 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

FrenchDeck 클래스를 통해 카드를 섞어보자. 표준 random.shuffle( ) 함수는 아래와 같이 사용하지만 FrechDeck 객체를 섞으려하면 [예제 11-5]와 같은 예외가 발생한다. 

In [7]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[1, 2, 4, 5, 0, 8, 9, 6, 3, 7]

In [8]:
from random import shuffle
l = list(range(10))
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

에러 메시지가 원인을 잘 설명하고 있다. 'FrenchDeck' 객체가 할당을 지원하지 않기 때문이다. shuffle() 함수는 컬렉션 안의 항목을 교환시킴으로써 작동하는데, FrenchDeck 클래스는 불변 시퀀스 프로토콜만 구현하고 있다. 가변 시퀀스는 \_\_setitem\_\_( ) 메서드도 지원해야 한다. 파이썬은 동적 언어이므로 코드를 대화형 콘솔에서 실행하는 동안에도 이 문제를 수정할 수 있다.

deck 객체에 \_cards라는 이름의 속성이 있고, \_cards가 가변 시퀀스임을 set_card()가 알고 있다는 것이 비결이다. 그러고 나서 set_card() 함수가 FrenchDeck 클래스의 \_\_setitem\_\_ 특별 에서드에 연결된다. 이 방법은 멍키 패칭<sup>monkey patching</sup>의 한 예다. 멍키 패칭은 소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위를 말한다. 멍키 패칭은 강력하지만, 비공개 속성이나 문서화되지 않은 부분을 다르는 경우가 많기 때문에 패치하는 코드와 패치될 프로그램이 아주 밀접하게 연관되어 있다. 

[예제 11-6]은 멍키 패칭의 사례를 보여주는 것 외에도 프로토콜이 동적이라는 것을 잘 보여준다. random.shuffle( ) 함수는 자신이 받는 인수의 자료형에 대해서는 신경쓰지 않는다. 단지 받은 객체가 일부 가변 시퀀스 프로토콜을 구현하고 있으면 될 뿐이다. 심지어 해당 객체가 필요한 메서드를 원래부터 가지고 있었는지, 아니면 나중에 얻었는지는 전혀 문제가 되지 않는다.

In [9]:
""" [예제 11-6] FrenchDeck을 가변형으로 만들어 random.shuffle()을 사용하기 위한 멍키 패칭 """

def set_card(deck, position, card): 
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card # 함수를 FrenchDeck 클래스의 __setitem__이라는 이름의 속성에 할당한다.
shuffle(deck)
print("  {}".format(deck[:5]))

from types import FunctionType
[x for x, y in FrenchDeck.__dict__.items() if type(y) == FunctionType] # __setitem__ 함수가 추가됨

  [Card(rank='9', suit='clubs'), Card(rank='4', suit='hearts'), Card(rank='A', suit='hearts'), Card(rank='3', suit='hearts'), Card(rank='10', suit='diamonds')]


['__init__', '__len__', '__getitem__', '__setitem__']

### 11.4 알렉스 마르텔리의 물새

구스 타이핑이라는 말은 cls가 추상 베이스 클래스인 경우, 즉 cls의 메타 클래스가 abc.ABCMeta인 경우에는 isinstance(obj, cls)를 써도 좋다는 것을 의미한다. 이미 collections.abc 및 파이썬 표준 라이브러리 numbers 모듈에서 유용한 추상 클래스를 많이 찾아볼 수 있다. 

구상 클래스에 비해 ABC가 가진 여러 개념적 장점 중 파이썬의 ABC에 실제 아주 큰 도움이 되는 register()라는 클래스 메서드가 있다. 이 메서드는 어떤 클래스가 ABC의 '가상' 서브 클래스임을 '선언'할 수 있게 해준다. 이렇게 선언하려면 등록할 클래스가 ABC의 메서드 이름 및 시그너처 요구사항을 만족해야 하며, 특히 메서드의 의미를 지켜야 한다. 그러나 ABC를 염두에 두고 등록할 클래스를 개발할 필요는 없다. 이런 방식은 엄격함과 강력한 결합을 상당히 파괴하므로 대부분의 OOP 프로그래머가 일반적으로 사용하는 상속보다 훨씬 더 많은 주의를 요구한다. 

심지어 클래스를 ABC의 서브클래스로 인식시키기 위해 등록할 필요가 없는 경우도 있다. 이런 경우 ABC는 결국 몇 가지 특별 메서드로 축약된다. 

In [10]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

여기에서 볼 수 있는 것 처럼 abc.Sized 클래스는 Struggle을 일종의 서브 클래스로 인식한다. 단지 \_\_len\_\_()이라는 특별 메서드만 구현하면 되며, 등록할 필요도 없다. 다만 \_\_len\_\_() 메서드는 구문(인수를 받지 않는 콜러블)과 의미(객체의 '길이'를 나타내는 0이나 양의 정수를 반환)를 지켜야 한다. 어쨌든 구문과 의미를 적절히 지키지 않고 구현된 특별 메서드는 훨씬 더 많은 문제를 겪게 된다. 

이제 설명을 정리할 때가 되었다. numbers.collections.abc, 혹은 여러분이 사용할 다른 프로임워크에 있는 ABC가 표현하는 개념을 실현하는 클래스를 구현할 때는 언제나 해당 ABC를 상속하거나 해당 ABC에 등록하라. 이 과정을 빠뜨리고 클래스를 정의한 라이브러리나 프레임워크를 사용하는 프로그램에서는 언제나 코드 시작부분에서 여러분이 클래스를 직접 등록하길 바란다. 그러고 나서 예를 들어 인수가 '시퀀스'인지 검사해야 할 때는 다음과 같이 한다. 
```
isinstance(the_arg, collections.abc.Sequence)
```
그리고 배코용 코드에서 절대로 ABC나 메타 클래스를 직접 구현하지 말라. ABC를 구현하고 싶은 생각이 든다면, '멋진 방치를 새로 장만한 사람에게 모든 문제가 못으로 보이는 증세'라고 확신할 수 있다. 깊이를 억제하고, 지관적으로 단순한 코드를 고수한 덕분에 나중에 여러분 코드를 유지보수할 사람이 훨씬 더 행복할 것이다. 


[예제 11-7] 문자열이나 문자열의 반복 가능형을 처리하기 위한 덕 타이핑
```
# isinstance() 보다는 아래와 같은 방식으로 덕타이핑을 사용하자
try:
    field_names = field_names.replace(',', ' ').split() # 상속관계를 확인하기 보다는 실제 사용하면서 나오는 오류에 주목해야 한다.
except AttributeError:
    pass
field_names = tuple(field_names)
```

### 11.5 ABC 상속하기
파이썬은 모듈을 로딩하거나 컴파일할 때가 아니라, 실행 도중 실제로 FrenchDeck2 객체를 생성할 때 추상 메서드의 구현 여부를 확인한다. 이때 추상 메서드 중 하나라도 구현되어 있지 않으면 "추상 메서드 \_\_delitem\_\_(), insert()를 가진 추상 클래스 FrenchDeck2의 객체를 생성할 수 없습니다." 라는 메시지와 함께 TypeError 예외가 발생한다. 그렇기 때문에 우리가 구현한 FrenchDeck2 예제에서 사용하지 않는 \_\_delitem\_\_(), insert
() 메서드를 구현해야 했다.

또한, ABC로부터 상송한 메서드를 효율이 더 뛰어난 메서드로 오버라이드할 수도 있다. 예를 들어 \_\_contains\_\_()는 시퀀스 전체를 조사하지만, 구상 클래스가 항목들을 정렬된 상태로 유지하고 있다면 bisect() 함수 (2.8절 참조)를 이용해서 이진 검색함으로써 속도를 향상시킬 수 있다.

In [11]:
""" [예제 11-8] collections.MutableSequence의 서브클래스 FrenchDeck2 """

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spaceds diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value): # 카드를 섞기 위해서는 __setitem__() 메서드만 있으면 된다.
        self._cards[position] = value
        
    def __delitem__(self, position): # 그러나 MutableSequence 클래스를 상속했으므로, 이 클래의 추상 메서드인 __delitem__()도 구현해야 한다.
        del self.cards[position]
        
    def insert(self, position, value): # 그리고 MutableSequence의 세 번째 추상 메서드인 insert()도 구현해야 한다.
        self._cards.insert(position, value)

### 11.6 표준 라이브러리의 ABC
#### 11.6.1 collections.abc의 ABC
[그림 11-3]의 주요 부분을 요약하면 다음과 같다.
<ul>
    <li> Iterable, Container, Sized </li>
    모든 컬랙션은 이 ABC를 상속하거나, 적어도 호환되는 프로토콜을 구현해야 한다. Iterable은 __iter__()를 통해 반복을, Container는 __contains__()를 통해 in 연산자를, Sized는 __len__()을 통해 len() 메서드를 지원한다.
    <li> Sequence, Mapping, Set </li>
    주요 불변 컬렉션형으로서 각기 가변형 서브클래스가 있다.MutableSequence에 대한 자세한 다이어그램은 [그림 11-2]를, MutableMapping과 MutableSet에 대한 다이어그램은 [그림 3-1], [그림 3-2]를 참조하라
    <li> MappingView </li>
    파이썬 3에서 item(), keys(), values() 메서드에서 반환된 객체는 각기 ItemsView, KeysView, ValuesView를 상속한다. ItemsView와 ValuesView는 풍부한 인터페이스를 제공하는 Set을 상속하므로 3.8.3절 '집합 연산'에서 설명한 연산자들이 포함된다.
    <li> Callable, Hashable </li>
    이 두 ABC는 컬렉션과 밀접한 연관이 있는 것은 아니지만, collections.abc가 파이썬 표준 라이브러리 안에 ABC를 정의한 최초의 패키지며, 이 두 모듈은 여기에 포함시킬 가치가 있다고 생각했다. 이 두 클래스는 주로 어떤 객체를 호출하거나 해시할 수 있는지 안전하게 판단하기 위해 isintance() 함수와 함께 사용된다. ※ 호출 가능 여부를 판단하기 위해서는 내장된 callable() 함수를 사용할 수도 있지만, 이에 준하는 hashtable() 함수는 없기 때문에 isinstance(my_obj, Hashtable)을 사용한다.
    <li> Iterator </li>
    Iterator는 Iterable을 상속한다. 이에 대해서는 14장에서 자세히 설명한다.
    
</ul>


#### 11.6.2 ABC의 숫자탑
nubmer 패키지는 소위 숫자탑이라고 하는 것을 정의한다. 다음과 같이 Number가 최상위 슈퍼클래스를 시작으로 Intergral까지 내려간다.
<li> Number </li>
<li> Complex </li>
<li> Real </li>
<li> Rational </li>
<li> Intergral </li>

따라서 정수형인지 검사해야 하는 경우 isinstance(x, numbers.Integral)을 이용해서 int형, bool형(int형의 서브클래스), 또는 자신을 numbers ABC에 등록한 정수형을 받을 수 있다. 그리고 언제든 클래스를 isinstance(x, numbers.Integral) 검사를 통과할 수 있다. 

그러나 값이 실수형이 될 수 있고 isinstance(x, numbers.Real)로 검사하는 경우 bool, int, float, fractrions.Fraction, 또는 Numpy 등 외부 라이브러리에서 제공하는 복소수 외의 숫자형을 받을 수 있다. NumPy는 자신의 자료형을 적절히 등록한다.

※ decimal.Decimal은 numbers.Real의 가상 서브클래스로 등록되어 있지 않다. 프로그램 안에서 정밀도 높은 Decimal을 필요로 하는 경우 실수로 float처럼 정밀도가 떨어지는 숫자형과 Decimal이 섞이는 문제를 예방하기 위한 것이다.

※ 파이선에서 부동소수점 오차 해결하기 : https://winterj.me/Floating-Point/

### 11.7 ABC의 정의와 사용
<b> 가정 : 웹사이트나 모바일 앱에서 광고를 무작위 순으로 보여주어여 하지만, 광고 목록에 들어 있는 광고를 모두 보여주기 전까지는 같은 광고를 반복하면 안 된다. </b>

<p>Tombola ABC는 메서드를 네 개 가지고 있다. 그중 두 개의 추상 메서드는 다음과 같다. 
<li> load(): 항목을 컨테이너 안에 넣는다.
<li> pick(): 컨테이너 안에서 무작위로 항목 하나를꺼내서 반환한다.</p>

<p>나머지 두 개의 구상 메서드는 다음과 같다.
<li> loaded(): 컨테이너 안에 항목이 하나 이상 들어 있으면 True를 반환한다.
<li> inspect(): 내용물을 변경하지 않고 현재 컨테이너 안에 들어있는 항목으로부터 만든 정렬된 튜플을 반환한다.</p>

In [12]:
""" [예제 11-9] 추상 메서드 두 개와 구상 메서드 두 개를 가진 Tombola ABC """
import abc

class Tombola(abc.ABC): # ABC를 정의하려면 abc.ABD를 상속해야 한다.
    
    @abc.abstractmethod # 추상 메서드를 @abstractmethod 데커레이터로 표시한다.
    def load(self, iterable):
        """ iterable의 항목들을 추가한다. """
    
    @abc.abstractmethod
    def pick(self):
        """ 무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드 실행하면 'LookupError'가 발생한다.
        """
        
    def loaded(self):
        """ 최소 한 개의 항목이 있으면 True를 아니면 False를 반환한다. """
        return bool(self.inspect()) # 상당히 값비싼 연산을 수행한다. 
    
    def inspect(self):
        """ 현재 안에 있는 항목들로 구성된 정렬된 튜플을 반환한다. """
        items = []
        while True: # 골라낼 항목이 없는 경우 LookupError를 발생시키라고 문사화 문자열을 통해 코드 구현자에게 알려준다. 
            try:
                item.append(self.pick())
            except LookupError:
                break
                
        self.load(items) # pick()과 load() 메서드를 이용해서 항목들을 모두 꺼낸 후 다시 넣어서 Tombola 내부를 조사한다.
        return tuple(sorted(items))

#### 11.7.1 ABC 상세구문 (코딩 규칙)

일반적으로 누적된 함수 데커레이터의 순서는 중요하다. @abstractmethod 경우에는 다른 메서드 디스크립터와 함께 적용할 때는 이 데커레이터를 제일 안쪽에 위치시켜야 한다. 즉, @abstractmethod와 def 문 사이에는 어떤 것도 올 수 없다.

```
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    
    def an_abstract_classmethod(cls, ...):
        pass
```

#### 11.7.2 Tombola ABC 상속하기

[예제 11-12]의 BingoCage 클래스는 더 좋은 난수 생성기를 사용하도록 [예제 5-8]을 개선한 클래스이다.

In [21]:
""" [예제 11-12] Tombola의 구상 서브클래스 BingoCage """

import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom() # random.SystemRandom 클래스는 os.urandom() 함수를 기반으로 random API 구현한다.
                                                 # os.urandom() 함수는 '암호화에 적합한' 무작위 bytes를 생성할 수 있다. 
        self._items = []
        self.load(items) # 초기화를 load에 위임한다.
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items) # 평범한 random.shuffle() 함수 대신, SystemRandom 객체의 shuffle() 메서드를 사용한다.
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick() # [예제 5-8], Tombola 인터페이스와 무관하지만 해로운 것은 없다. 

[예제 11-13]은 Tombola 인터페이스를 제대로 구현하지만 아주 다른 클래스를 보여준다. '공'을 섞고 마지막 공을 꺼내는 대신 LotteryBlower는 임의의 위치에 있는 공을 꺼낸다.

In [14]:
""" [예제 11-13] Tombola의 inspect()와 loaded() 메서드를 오버라이드하는 Lotteryblower 구상 서브클래스 """

import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable) # 초기화 메서드는 어떤 반복형도 받을 수 있다. 
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls)) # 범위가 비어 있을 때 ValueError를 발생시키므로 Tombola 인터페이스를 따르기 위해
                                                          # ValueError를 잡고 대신 LookupError를 발생시킨다. 
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position) # 무작위로 선택한 공을 꺼낸다.
    
    def loaded(self):
        return bool(self._balls) # inspect()를 호출하지 않도록 loaded() 메서드를 오버라이드 한다. self._balls를 이용하면 속도를 향상시킨 수 있다. 
    
    def inspect(self): # 한 줄 짜리 코드로 inspect()를 오버라이드 한다. 
        return tuple(sorted(self._balls))

### 11.7.3 Tombola의 가상 서브클래스
구슾 타이핑의 본직적인 기능(그리고 물새 이름을 가질 수 있는 이유)은 어떤 클래스가 ABC를 상속하지 않더라도 그 클래스의 <b>가상 서브클래스</b>로 등록할 수 있다는 것이다. 이렇게 함으로써 이 클래스가 ABC에 정의된 인터페이스를 충실히 구현한다고 약속하는 것이다. 그리고 파이선은 검사하지 않고 우리를 믿어준다. 그러나 우리가 거짓말을 하면 런타임 예외가 발생한다.

ABC의 register() 메서드를 호출하면 클래스가 등록된다. 등록된 클래스는 ABC의 가상 서비클래스가 되어 issubclass()와 isinstance() 함수에 의해 인식되지만, ABC에서 상속한 메소드나 속성은 전혀없다. 

일반적으로 register() 메서드는 평범한 함수처럼 호출되지만 데커레이터로 사용할 수도 있다. [예제 11-14]에서는 데커레이터 구문을 이용해서 Tombola의 가상 서브클래스인 TomboList를 구현한다.

In [15]:
""" [예제 11-14] Tombola의 가상 서브클래스 TomboList """

from random import randrange
#from tombola import Tombola

@Tombola.register
class TomboList(list): # list를 상속한다. 
    
    def pick(self):
        if self: #TomboList는 list 에서 __bool__을 상속한다. 리스트가 비어 있지 않으면 True를 반환한다.
            position = randrange(len(self))
            return self.pop(position) # pick() 메서드는 무작위 인덱스를 전달해서 list에서 상속한 self.pop()을 호출한다.
        else:
            raise LookupError('pop from empty TomboList')
        
        load = list.extend
        
        def loaded(self): # bool 함수에 위임한다. 
            return bool(self)
        
        def inspect(self):
            return tuple(sorted(self))
        
    # Tombola.register(TomboList) # 파이썬 3.3 이하 버전에서는 register()를 클래스 데커레이터로 사용할 수 없다. 표준적인 호출구문을 사용해야 한다.

In [16]:
issubclass(TomboList, Tombola) # TomboList를 Tombola 클래스의 가상 서브 클래스로 등록했기 때문에 issubclass()와 isinstance() 함수는  
                               # TomboList가 Tombola의 서브클래스인 것처럼 판단한다.

True

In [17]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [18]:
TomboList.__mro__ # 상속은 메서드 결정순서를 담은 __mro__라는 특별 클래스 속성에 의해 운영된다. TomboList는 Tombola에서 아무런 메서드도 상속하지 않는다.

(__main__.TomboList, list, object)

### 11.8 Tombola 서브클래스 테스트 방법

Tombola 클래스를 테스트하기 위해 사용한 스크립트는 클래스 계층구조를 조사할 수 있게 해주는 다음과 같은 두 가지 클래스 속성을 사용한다.

<li>__subclasses__( )</li>
클래스의 바로 아래 서브클래스의 리스트를 반환하는 메서드, 리스트에 가상 서브클래스는 들어가지 않는다. 

<li>_abc_registry( )</li>
ABC에서만 사용할 수 있는 데이터 속성으로, 추상 클래스의 등록된 가상 서브클래스에 대한 약한 참조를 담고 있는 weakSet이다. 

In [19]:
""" [예제 11-15] Tombola 서브클래스에 대한 테스트 실행 코드 (결과는 tombola_runner.py 참조)"""
""" Jupyter notebook에서 실행되지 않음 """

import doctest

# 테스트할 모듈 
import bingo, lotto, tombolist, drum

TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

def main(args):
    verbose = '-v' in param
    # 메모리에 살아 있는 직속 서브클래스들을 나열한다. 그렇기 때문에 소스 코드안에서 참조하지 않으면서도 단지 클래스를 메모리에 로딩하기 위해 테스트할 모듈을 임포트하였다.
    real_subclasses = Tombola.__subclasses__()
    # __subclasses__()의 결과와 연결할 수 있도록 WeakSet인 _abc_registry 속성에서 list를 생성한다. 
    virtual_subclasses = list(Tombola._abc_registry)
    
    # 발견한 서브클래스들을 반복하면서 각 클래스 test() 함수에 전달한다.
    for cls in real_subclasses + virtual_subclasses:
        test(cls, verbose)
        
def test(cls, verbose=False):
    # 테스트할 클래스인 cls 인수는 doctest를 실행하기 위해 제공한 전역 네임스페이스에 있는 ConcreteTombola라는 이름에 바인딩된다. 
    res = doctest.testfile(
        TEST_FILE,
        globs={'ConcreteTombola':cls},
        verbose=verbose,
        optionflags=doctest.REPORT_ONLY_FIRST_FAILURE
    )
    
    # 클래스명, 시도한 테스트 횟수, 실패한 테스트 횟수, 'OK', 'FAIL' 라벨을 이용해서 테스트 결과를 출력한다.
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))
    
if __name__ == '__main__':
    import sys
    main(args)

ModuleNotFoundError: No module named 'tombola'